# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner and combine it with our other data skills. In this lab you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer follow-up questions. 


### Outline:

1. Determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create Python functions to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results.

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- What are the 5 businesses with the highest average ratings?
- What are the 5 businesses with the lowest average ratings?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- How many businesses have a rating above 4.5?
- How many businesses have a rating below 3?
- Return the text of the oldest review in the table.
- Return the overall rating of the business with the oldest review. 
- Find the highest rated business and return text of the newest review of the three.
- Find the lowest rated business and return text of the newest review of the three.  


## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [37]:
import mysql.connector
from mysql.connector import errorcode

# Connect to DB server on AWS

# Get the hostpost and credentials from python file saved on the same folder
import credentials

# Connect to the database using 'connect()' method
# Use parameters from python file imported earlier
cnx = mysql.connector .connect(
    host = credentials.host,
    user = credentials.user,
    passwd = credentials.password
)

print(cnx) # Test a connection object was created

# create a cursor
my_cursor = cnx.cursor()

In [38]:
## Create new DB 

# create a function to create a new DB
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)
        
# Create the DB for the businesses

db_name = 'yelp_lab'
try:
    my_cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(my_cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

Database yelp_lab does not exists.
Database yelp_lab created successfully.


In [39]:
# Create a table for the Businesses and for the reviews

# Add the text for the queries into a dictionnary
TABLES = {}
TABLES['bars'] = (
    "CREATE TABLE bars ("
    "  bar_id varchar(50) NOT NULL,"
    "  bar_name varchar(120),"
    "  price varchar(10),"
    "  rating float,"
    "  review_count int(10),"
    "  PRIMARY KEY (bar_id)"
    ") ENGINE=InnoDB")

TABLES['bar_review'] = (
    "CREATE TABLE bar_review ("
    "  key_review varchar(50) NOT NULL,"
    "  text_review varchar(50),"
    "  date_review date,"
    "  bar_id varchar(50) NOT NULL,"
    "  PRIMARY KEY (key_review),"
    "  CONSTRAINT FOREIGN KEY (bar_id) "
    "     REFERENCES bars (bar_id) ON DELETE CASCADE"
    ") ENGINE=InnoDB")


In [40]:
#Connect to DB - to be a function in another file later

cnx = mysql.connector.connect(
    host = credentials.host,
    user = credentials.user,
    passwd = credentials.password,
    database = db_name
)
my_cursor = cnx.cursor()

In [41]:
# Run the script for creating a table for the reviews

for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        my_cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

my_cursor.close()
cnx.close()

Creating table bars: OK
Creating table bar_review: OK


## Part 2: Create ETL pipeline for the business data from the API

In [12]:
# import clientID and key for yeld
# import api_key

In [42]:
client_id = "oXRz7HWebFp6YdQ0YT3PMQ"
api_key = "Mm2MVQluEx7HWKjAMxJQT_KOSB_nKvEeQmjn19VS9PnsOBDoqtegFU1RoBXG0Lum415Eus2bKYuo-eb6mgMm1xV5nhwh3H5uhg-HJN39VFSMjIkwyRCbpLVv7ws7XXYx"

In [43]:
# # write a function to make a call to the API
import time
import requests
import pandas as pd
 

def yelp_call(url_params, api_key):

    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()['businesses']
    return data

# print(data[0])

def all_results(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results.append(yelp_call(url_params, api_key))
        # function for getting specific variable and load to AWS can be used here
        time.sleep(1) #Wait a second
        cur += 50
    return results

term = 'bars'
location = 'New York'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }

all_bars = all_results(url_params, api_key)
print(len(all_bars))

all_bars[0][0]
df = pd.DataFrame(all_bars)
df.head()

6300 total matches found.
20


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,"{'id': 'H8FodkT7iwLR69gNJiES7Q', 'alias': 'bar...","{'id': 'xEF3Kvd0yw74pjnlFB2Sgg', 'alias': 'nob...","{'id': 'RemvTpxalDjSew5HwmNzUQ', 'alias': 'the...","{'id': 'xZwqtw3-dd6yJyixM-BYIA', 'alias': 'the...","{'id': 'zWrZBXj5F2V_0UknnTfqbw', 'alias': 'the...","{'id': 'AydiQxXWQLqVrZ9sEZTxtw', 'alias': 'the...","{'id': 'iM3zYjjmnJ06VVmxvTo9eQ', 'alias': 'nit...","{'id': '2DJjJu3sqaBCN7w4CwR5Dw', 'alias': 'liv...","{'id': '5Fz1WlY3lxy8gJFs8s_1Dw', 'alias': 'the...","{'id': 'OgM6WwztLVavBQBmb_g5aw', 'alias': 'tar...",...,"{'id': 'SBkebBPuX2mRUQ9VaQ91NQ', 'alias': 'boo...","{'id': 'aiRctIzCj9J3UqBHd-Z3HQ', 'alias': 'ple...","{'id': 'ePqKbqXFBCbwBaWiN2Jo9w', 'alias': 'amp...","{'id': '1VFAJtFoeTSwGCXVWBQEIg', 'alias': 'pub...","{'id': 'GyaLMW2fTB2oXmzhRqwfeg', 'alias': 'las...","{'id': '9DPI1WLAOfpiMXxHeQ0swg', 'alias': 'tai...","{'id': '6ZIGSGKlc16hR9JUGTYjNA', 'alias': 'the...","{'id': 'a8CdwY4uFIP4nXkl4V5EEg', 'alias': 'lit...","{'id': 'TqumO9YCFsKv3plJRsyP0A', 'alias': 'the...","{'id': 'Rc1lxc5lSKJYd162JHNMfQ', 'alias': 'bea..."
1,"{'id': 'YL1LxYufuwX5sq4RwW-2Gw', 'alias': 'flo...","{'id': '-dQexkmhlgbiKvZ94jBmjQ', 'alias': 'dea...","{'id': 'fG9pK2l_1A4P6wiFr47k-w', 'alias': 'the...","{'id': 'J3NT61-AH5d5Gu5tFJhYSw', 'alias': 'the...","{'id': 'Mf6txI5sAs9K7BE24Uazgw', 'alias': 'hou...","{'id': 'rRPuX18uiYLEUGdJ9xQBEQ', 'alias': 'zom...","{'id': 'ph-GkrDhM1pBwbs-pfvV1A', 'alias': 'gar...","{'id': 'J4pAppaiXwCkbLOGc4wHww', 'alias': 'rai...","{'id': '0itsOZxeHy2KPnA40Rwexw', 'alias': 'swe...","{'id': 'CM8WhuCoXd3j3u4Jjqol-g', 'alias': 'sav...",...,"{'id': 'kLEx49YQZZYhtf57YEvQIg', 'alias': 'bas...","{'id': 'Xj21kg8084FZd239fkkS-Q', 'alias': 'the...","{'id': 'iBv3jR8lxH2e9dV42Sca0g', 'alias': 'sup...","{'id': '9jceg0585cKrwBLarT9ftQ', 'alias': 'our...","{'id': '6qnWAPjJ7UER4QLCtotyBw', 'alias': 'amo...","{'id': '3R8vTK-WYzAmbAT-y4jOPw', 'alias': 'bab...","{'id': 'MfoJDhRzT6ZDhK4mvTWLJQ', 'alias': 'har...","{'id': '3lLBRLbXAIgd1yZgWgnfeg', 'alias': 'dut...","{'id': 'hKChLzaG8tyrSrg3U_wRNw', 'alias': 'thr...","{'id': '_l40VhhS3pNm55yZTLgLuw', 'alias': 'vel..."
2,"{'id': 'bRzE-FnE1QgbUb-3cosoUA', 'alias': 'don...","{'id': 'xprzjm5svSSUCZOnaUbQDQ', 'alias': 'jim...","{'id': 'P9x0uX9nmPCph1mWM3k4vg', 'alias': 'bar...","{'id': 'qfF0QWdKd3YDPPLThyB8UQ', 'alias': 'clo...","{'id': 'ebAV5BF2JvObg9tDPZ5Hwg', 'alias': 'til...","{'id': 'BHpAlynD9dIGIaQDRqHCTA', 'alias': 'sun...","{'id': 'VBjYIelQBlCK889IBVrWzQ', 'alias': 'pat...","{'id': 'sst-eDliHsxUJ8cNyBriHw', 'alias': 'win...","{'id': 'DB45nM2VMUocjPejFArc1g', 'alias': 'the...","{'id': '9itMlJoBaBDGjqgnyDem-w', 'alias': '142...",...,"{'id': 'nCH7SR7qXu0asO1GU3iqHw', 'alias': 'the...","{'id': 'EDOUZDqwvD4UzOy0doER0g', 'alias': 'jun...","{'id': 'jEEb_EKKD2lx2mvTly_U6Q', 'alias': 'wat...","{'id': 'ODjhHdynpv4g9y0lmKvaiA', 'alias': 'sky...","{'id': 'f7g3GhoQs9nQVTJEL2p48Q', 'alias': 'gra...","{'id': 'l8um5IoiMqV5RGgtFGwOkA', 'alias': 'clo...","{'id': 'XLT7aNLJEhIdoieNjPw8Gg', 'alias': 'blu...","{'id': 'dYyd66STjtwdO9_02SgLDg', 'alias': 'con...","{'id': '_8Oq6MmhqLgzQDVYXWLxuA', 'alias': 'réu...","{'id': '-5oK5IO9aPrAKuTf0kKpDg', 'alias': 'lem..."
3,"{'id': 'w83h9sA6mnNEGHMu_Wm67Q', 'alias': 'fig...","{'id': '89XYJZSPKt-dqjE1zyTIzQ', 'alias': 'bat...","{'id': 'AGIaARDApOvSC1toWSOLuw', 'alias': 'zig...","{'id': 'bQraKjILxZGN1GNKt5XUKA', 'alias': 'ram...","{'id': 'TE87BLxClCcIg62i0xVgMg', 'alias': 'moo...","{'id': '2_1FxCMLvK46Vt81VlIS7A', 'alias': 'min...","{'id': 'omZBT1AXN5XkHPR6Kbie9g', 'alias': 'kil...","{'id': 'L9RENH-3lzR3RGvc-qcSqA', 'alias': '40-...","{'id': 'bEloKDe-QVkwakAwmFKbbA', 'alias': 'fry...","{'id': '8ucQ3A7f6h9_PkKStYBkQA', 'alias': 'lal...",...,"{'id': 'fmG6BTunLXI2cN61NuLHiw', 'alias': 'oph...","{'id': 'ne1sjzU07jRQZSFO0K9l9Q', 'alias': '68-...","{'id': 'XO3yD87rTPz9o6Dy9qRaJg', 'alias': 'the...","{'id': 'svYBY_pf7p2VezMyWZtKAg', 'alias': 'wea...","{'id': 'pcb8yK9_28uWz5CyokjwSg', 'alias': 'ban...","{'id': 'VLGmFHE4s0Sf

In [44]:
#function to parse the data and keep only the ones to be uploaded in AWS DB
import mysql.connector
from mysql.connector import errorcode

def parse_api(data):
    parsed_data = []
    final_parsed_bars = []
    for i in range(len(data)):
        for j in range(50):
            parsed_data.append(data[i][j])
    
    #create a list with SQL tables columnns names
    for parsed_bar in parsed_data:
        if 'price' in parsed_bar.keys():
            final_parsed_bars.append({'bar_id': parsed_bar['id'],'bar_name': parsed_bar['name'],
                         'price': parsed_bar['price'],'rating': parsed_bar['rating'],
                         'review_count': parsed_bar['review_count']})
        else:
            final_parsed_bars.append({'bar_id': parsed_bar['id'],'bar_name': parsed_bar['name'],
                         'price': "",'rating': parsed_bar['rating'],
                         'review_count': parsed_bar['review_count']})
    return final_parsed_bars

#call the function and return a list with only the columns names in SQL
final_parsed_bars = parse_api(all_bars)

#format the businesses names for inserting
for parsed_bar in final_parsed_bars:
    parsed_bar['bar_name'] = parsed_bar['bar_name'].replace("'"," ")
    
final_parsed_bars


[{'bar_id': 'H8FodkT7iwLR69gNJiES7Q',
  'bar_name': 'Bar Wayo',
  'price': '',
  'rating': 5.0,
  'review_count': 4},
 {'bar_id': 'xEF3Kvd0yw74pjnlFB2Sgg',
  'bar_name': 'Nobody Told Me',
  'price': '',
  'rating': 5.0,
  'review_count': 28},
 {'bar_id': 'RemvTpxalDjSew5HwmNzUQ',
  'bar_name': 'The Dead Rabbit',
  'price': '$$',
  'rating': 4.0,
  'review_count': 1459},
 {'bar_id': 'xZwqtw3-dd6yJyixM-BYIA',
  'bar_name': 'The Pub',
  'price': '££',
  'rating': 4.5,
  'review_count': 2},
 {'bar_id': 'zWrZBXj5F2V_0UknnTfqbw',
  'bar_name': 'The Up & Up',
  'price': '$$',
  'rating': 4.0,
  'review_count': 400},
 {'bar_id': 'AydiQxXWQLqVrZ9sEZTxtw',
  'bar_name': 'The Ship',
  'price': '$$',
  'rating': 4.0,
  'review_count': 316},
 {'bar_id': 'iM3zYjjmnJ06VVmxvTo9eQ',
  'bar_name': 'Nitecap',
  'price': '$$',
  'rating': 4.0,
  'review_count': 314},
 {'bar_id': '2DJjJu3sqaBCN7w4CwR5Dw',
  'bar_name': 'Livingston Manor',
  'price': '$$',
  'rating': 4.0,
  'review_count': 111},
 {'bar_id'

In [45]:
#function to remove duplicates

def remove_duplicate(dup_list):
    seen = set()
    no_dup_list = []
    for the_dict in dup_list:
        test = tuple(the_dict.items())
        if test not in seen:
            seen.add(test)
            no_dup_list.append(the_dict)
    return (no_dup_list)

#call function on list of bars to remove duplicates
final_unique_parsed_bars = remove_duplicate(final_parsed_bars)
final_unique_parsed_bars

[{'bar_id': 'H8FodkT7iwLR69gNJiES7Q',
  'bar_name': 'Bar Wayo',
  'price': '',
  'rating': 5.0,
  'review_count': 4},
 {'bar_id': 'xEF3Kvd0yw74pjnlFB2Sgg',
  'bar_name': 'Nobody Told Me',
  'price': '',
  'rating': 5.0,
  'review_count': 28},
 {'bar_id': 'RemvTpxalDjSew5HwmNzUQ',
  'bar_name': 'The Dead Rabbit',
  'price': '$$',
  'rating': 4.0,
  'review_count': 1459},
 {'bar_id': 'xZwqtw3-dd6yJyixM-BYIA',
  'bar_name': 'The Pub',
  'price': '££',
  'rating': 4.5,
  'review_count': 2},
 {'bar_id': 'zWrZBXj5F2V_0UknnTfqbw',
  'bar_name': 'The Up & Up',
  'price': '$$',
  'rating': 4.0,
  'review_count': 400},
 {'bar_id': 'AydiQxXWQLqVrZ9sEZTxtw',
  'bar_name': 'The Ship',
  'price': '$$',
  'rating': 4.0,
  'review_count': 316},
 {'bar_id': 'iM3zYjjmnJ06VVmxvTo9eQ',
  'bar_name': 'Nitecap',
  'price': '$$',
  'rating': 4.0,
  'review_count': 314},
 {'bar_id': '2DJjJu3sqaBCN7w4CwR5Dw',
  'bar_name': 'Livingston Manor',
  'price': '$$',
  'rating': 4.0,
  'review_count': 111},
 {'bar_id'

In [46]:
# Write a function to take your parsed data and insert it into the DB
import mysql.connector
from mysql.connector import errorcode

cnx = mysql.connector.connect(
    host = credentials.host,
    user = credentials.user,
    passwd = credentials.password,
    database = db_name
)
my_cursor = cnx.cursor()

# add_bars = ("INSERT INTO bars "
#                "(bar_id, bar_name, price, rating, review_count) "
#               "VALUES (%(bar_id)s, %(bar_name)s, %(price)s, %(rating)s), %(review_count)s)")
# add_salary = ("INSERT INTO salaries "
#               "(emp_no, salary, from_date, to_date) "
#               "VALUES (%(emp_no)s, %(salary)s, %(from_date)s, %(to_date)s)")


for line in final_unique_parsed_bars:
    cnx = mysql.connector.connect(
    host = credentials.host,
    user = credentials.user,
    passwd = credentials.password,
    database = db_name
    )
    my_cursor = cnx.cursor()
    
    print(f"INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('{line['bar_id']}', '{line['bar_name']}', '{line['price']}', '{line['rating']}', '{line['review_count']}')")
    my_cursor.execute(f" INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('{line['bar_id']}', '{line['bar_name']}', '{line['price']}', '{line['rating']}', '{line['review_count']}'); ")
    cnx.commit()

INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('H8FodkT7iwLR69gNJiES7Q', 'Bar Wayo', '', '5.0', '4')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('xEF3Kvd0yw74pjnlFB2Sgg', 'Nobody Told Me', '', '5.0', '28')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('RemvTpxalDjSew5HwmNzUQ', 'The Dead Rabbit', '$$', '4.0', '1459')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('xZwqtw3-dd6yJyixM-BYIA', 'The Pub', '££', '4.5', '2')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('zWrZBXj5F2V_0UknnTfqbw', 'The Up & Up', '$$', '4.0', '400')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('AydiQxXWQLqVrZ9sEZTxtw', 'The Ship', '$$', '4.0', '316')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('iM3zYjjmnJ06VVmxvTo9eQ', 'Nitecap', '$$', '4.0', '314')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('2DJ

INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('I4LSdJd2Z8M6VNLhbX3aLA', 'Superfine', '$$', '3.5', '437')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('2iW5TU4W0BN8LYBzVmO9Qw', 'The Honey Well', '$$', '4.5', '134')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('XyulGZ5P9VhtNcK7zN2R9g', 'Cardiff Giant', '$$', '4.0', '48')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('cGOuBiTCo5ZuyU9v4zZezw', 'Boat Bar', '$', '4.0', '147')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('MBkyCEjBbe-Ql63agwOUpw', 'Duff s Brooklyn', '$', '4.5', '158')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('zyxI5ko5Dpkqi9q5oSGa9w', 'Pilot', '$$$', '3.5', '91')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('-9MLyUR2SyI2zXBBvksj3A', 'Jackdaw', '', '5.0', '28')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('

INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('SaYTVG7wHCMDBofHCgXPaA', 'Dutch Freds', '$$', '4.5', '1315')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('vYGi3439teYg1wPzqVcHUA', 'Grand Banks', '$$$', '4.0', '395')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('ZvwTeYdfe6QqbtmrV0I07Q', 'The Rockwell Place', '', '4.5', '11')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('MVyhLMe2xgM2ndcwIkyQkw', 'Featherweight', '$$', '4.5', '128')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('oBcuOhubYwHV_bY35lu5hA', 'Petee s Cafe', '$$', '4.5', '70')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('KMxoDwzlpa_PTw3AoGfq2w', 'Johnny s Bar', '$', '4.0', '237')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('m1SprA3LBB5mL8QaH5GTew', 'Lone Wolf', '$', '4.0', '96')
INSERT INTO bars (bar_id, bar_name, price, rating, review_

INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('xHVQHe0vhtIPPtK9wUqb7A', 'Jackbar', '$', '4.5', '106')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('vyRQGK3qABg1HSWNpMbmtg', 'The Cabinet', '', '5.0', '7')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('WoAvunLY0_L0ix8Vgit8gQ', 'The Tuck Room', '$$', '4.0', '231')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('7uJBW357VM1x57vBbILrgQ', 'Clandestino', '$$', '4.0', '103')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('DQV4cwauKsK_RJnhaqGAzQ', 'Lion Lion', '$$', '4.5', '75')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('XipQLDbyTl5tsLlyzAWzug', 'Jajaja', '$$', '4.5', '729')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('fmG6BTunLXI2cN61NuLHiw', 'Ophelia', '$$$', '4.0', '105')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('ne1sjz

INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('11dPQDW_c8Y3F0yhDYBXvA', 'The Polynesian', '$$', '4.0', '255')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('okF1w-7gZNy2OB73SsjuYw', 'Travel Bar', '$$', '4.5', '43')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('BPbpVx_U-J5vo3joig7oBg', 'Honore Club', '', '5.0', '7')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('KqxBfDaww1j0qhgDGh-VSg', 'Diamond Dogs', '$$', '4.0', '124')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('WaUuOzaY7WnmBer2nTLPUQ', 'Mace', '$$', '4.5', '160')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('nUTf2hcDBv4xnbLaIkhV5g', 'Loosie Rouge', '$$', '4.0', '76')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('uVimnyjMcc7MOGzd2NRX-w', 'North Pole Pub', '$$', '5.0', '34')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALU

...

INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('KFKxR3w5XlGFYB1OhW1SUA', 'Mirrors On Grand', '$', '4.0', '22')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('4KqGAtsXe7FXDRvnRQCjVA', 'Pioneers', '$$', '3.5', '252')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('mWQiNb6IUGqIRLzR3rE-Fg', 'Burke & Wills', '$$$', '4.0', '395')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('8Wk7ciMcn3rPASMmN3tF_w', 'Barcade', '$$', '3.5', '382')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('BglFB3mkuo_PVtxdok4v3A', 'Vinegar Hill House', '$$$', '3.5', '640')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('WaNdisNSGLNzn_epZAtHPA', 'The Windjammer', '$', '4.5', '7')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('Hgc27X2PX2jZQMcVtbJ9vQ', 'Mother of Pearl', '$$', '4.0', '390')
INSERT INTO bars (bar_id, bar_name, price, rating, 

INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('QA0FUxP1US5LIyf2tG5KUw', 'Illusions Hookah & Mixology Lounge', '$$', '3.5', '94')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('Gwi51JUxbKT1Eg-C3JrxRQ', 'La Loba Cantina', '$$', '4.0', '101')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('sYhOOLd04h7dfaM9q49w9A', 'The Bell House', '$$', '4.0', '373')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('Qm6oQkgosT-pcEmlo4i5Gw', 'Holland Bar', '$', '4.0', '102')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('C8K3kTQ2NX5mm4jzQBRVUg', 'Dive 75', '$$', '4.0', '266')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('jjLcqGI11ThxAhcftEh1Aw', 'Fraunces Tavern', '$$', '3.5', '813')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('6u-4kVhNhpXuRjTO426eag', 'Fifth Hammer Brewing', '$$', '4.5', '101')
INSERT INTO bars (bar_id,

INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('fxKrQnF7zzGwaHiTHvWyBQ', 'Call Box Lounge', '$', '3.5', '29')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('DCTWsTYqkbwJ-pFCIwmHew', 'Gleason s Tavern', '$$', '4.0', '118')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('PldbjkxtHmWC2QVyV5dAbQ', 'Wilfie & Nell', '$$', '3.5', '344')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('fD8eRYGUPQkxjFs9AUF6-w', 'Halsey Bar And Grill', '$$', '4.0', '35')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('0EtZ1xhnXWp4-ysNVwItYA', 'Distilled', '$$', '4.0', '757')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('s-n0KW8iqiq-C96tpxm2rA', 'The Hairy Lemon Pub', '$', '4.5', '51')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('ZTsN8TdMXl8SW_AKd_ablw', 'Forno Rosso', '$$', '4.0', '637')
INSERT INTO bars (bar_id, bar_name, price

INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('cOUYYcrOeManYPyuun_xVg', 'Jimbo Slims', '', '5.0', '32')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('rP-5Z9MsEqtiENCiPqsizg', 'Monkey Bar', '$$$', '3.5', '271')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('Lr1KN7oOhiL5Hd2yikpIgA', 'The Hunterian Bar', '', '4.5', '12')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('0yg9DS4RDceQ6HgH3Jx80Q', 'Sean s Bar & Kitchen', '$$', '4.0', '253')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('0gTHz_iR8yHuhtIt-0-20g', 'Zum Schneider', '$$', '4.0', '574')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('qtFJ6T6-bO7IhOpgWb7XLg', 'Spuyten Duyvil', '$$', '4.0', '376')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('hu2bo4hZueedSKBimTAqOw', 'Humboldt and Jackson', '$$', '4.5', '116')
INSERT INTO bars (bar_id, bar_name, pri

INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('CcxreLJbZsO6Z775-hMWpw', 'St Tropez Wine Bar', '$$', '4.5', '118')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('OWk0cK23jsgDErVl7zOd7Q', 'Paddy Reilly s Music Bar', '$$', '4.0', '91')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('e64Y17DJgjotSz8B9gEI7Q', 'Ugly Baby', '$$', '4.5', '271')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('VP_HZr_j0dXKd80lUBfsyg', 'The Wicked Monk', '$$', '4.0', '221')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('Qh5cVgmoX09OMFSDtri7Vg', 'House of Brews', '$$', '4.0', '504')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('Z6BM0HayUS5tHW41DrxtbA', 'Oracle Lounge', '', '3.5', '7')
INSERT INTO bars (bar_id, bar_name, price, rating, review_count) VALUES ('IzpoR2NGsTJPgpAIYKq4Rg', 'The Graham', '$', '4.0', '76')
INSERT INTO bars (bar_id, bar_name, price, 

In [28]:
#Commit data to the database
len(final_parsed_bars)
my_cursor.execute("""DROP DATABASE yelp_lab""")

## Part 3: Create ETL pipeline for the review data from the API

In [47]:
# write a query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant
all_business_id = []
for parsed_bar in final_parsed_bars:
    all_business_id.append(parsed_bar['bar_id'])

all_business_id

['H8FodkT7iwLR69gNJiES7Q',
 'xEF3Kvd0yw74pjnlFB2Sgg',
 'RemvTpxalDjSew5HwmNzUQ',
 'xZwqtw3-dd6yJyixM-BYIA',
 'zWrZBXj5F2V_0UknnTfqbw',
 'AydiQxXWQLqVrZ9sEZTxtw',
 'iM3zYjjmnJ06VVmxvTo9eQ',
 '2DJjJu3sqaBCN7w4CwR5Dw',
 '5Fz1WlY3lxy8gJFs8s_1Dw',
 'OgM6WwztLVavBQBmb_g5aw',
 'uUPs_2S54rROkRTjlDOJVg',
 'P3NfoMjBy_o2BTNNZ-5okg',
 '511OTzlr0FkL4esfBxMzIw',
 'gjwELZmHUNO0WdqXYBEbcg',
 'H8mnryiCI02N00g9vuFDLg',
 'H7S7-vF2z_HfaLo6BGPetw',
 'eTR6nfHvQzvKX7Dcd6JrfA',
 'zvxUXRQOqzsyFiCKS09_JQ',
 'c3Ww3pgeHjBi3ctkeUWrjQ',
 'e08qVeVYBR-0QI_G0NDZpw',
 'NVx7gfdd4NDAwvj6URZ2aQ',
 '_Tn0eN5E37WBf8oVdOA9BA',
 '9sU8ceKHiqNrLBMRM-aUkg',
 '6awY1cqM1DIQ0h8EJvs2fQ',
 'X2aj0e6EL2-Q3SX8eABenA',
 'bkIMbvXOcztDVz8bpyhMzQ',
 'H1QphdrrsaB472y9HXi2Ow',
 'HtvftOs_PPK3wo2Fgopv4w',
 '0RSyqMHlkGEz9eoXpbIKow',
 'X_2RR5F7Bq3jos28dQuvlg',
 'ZNEFXERBvl7i_QQojRoKNQ',
 'PszZvrDqzgPpl6SQuk3K9Q',
 'PcF_ebeELn7mg6vqVnHjIg',
 '-egltOQhAdo1kWlPSJfmvA',
 'd6mslo1LlpkNPgb2HWOlEg',
 '204fu_NmR6TK_s1FMXN6og',
 'ZD22cMZvSmfNsXS9vKizGA',
 

In [56]:
# write a function that take a business id 

# and makes a call to the API for reivews
# then parse out the relevant information

def get_url_review(list_id,api_key):
    url_temp = 'https://api.yelp.com/v3/businesses/{id}/reviews'
    list_urls = []
    for x in list_id:
        url_review = url_temp.replace("{id}",x)
        headers = {'Authorization': 'Bearer {}'.format(api_key)}
        response = requests.get(url_review, headers=headers)
        data = response.json()['reviews']
        list_urls.append(data)
    return list_urls
    

# def all_results(list_url, api_key):
#     url = 'https://api.yelp.com/v3/businesses/search'
#     headers = {'Authorization': 'Bearer {}'.format(api_key)}
#     response = requests.get(url, headers=headers, params=url_params)
#     num = response.json()['total']
#     print('{} total matches found.'.format(num))
#     cur = 0
#     results = []
#     while cur < num and cur < 1000:
#         url_params['offset'] = cur
#         results.append(yelp_call(url_params, api_key))
#         # function for getting specific variable and load to AWS can be used here
#         time.sleep(1) #Wait a second
#         cur += 50
#     return results


# all_bars = all_results(url_params, api_key)
# print(len(all_bars))

# all_bars[0][0]
# df = pd.DataFrame(all_bars)
# df.head()

In [ ]:
get_url_review(all_business_id,api_key)

In [43]:
# write a function that take a business id 

# and makes a call to the API for reivews
# then parse out the relevant information

def get_url_review(list_id):
    url_temp = 'https://api.yelp.com/v3/businesses/{id}/reviews'
    list_url = []
    for x in list_id:
        list_url.append(url_temp.replace("{id}",x))
    return list_url
    
list_urls = get_url_review(all_business_id) 
list_urls
    
# def yelp_call(url_params, api_key):

#     url = 'https://api.yelp.com/v3/businesses/search'
#     headers = {'Authorization': 'Bearer {}'.format(api_key)}
#     response = requests.get(url, headers=headers, params=url_params)
#     data = response.json()['businesses']
#     return data

# def all_results(list_url, api_key):
#     url = 'https://api.yelp.com/v3/businesses/search'
#     headers = {'Authorization': 'Bearer {}'.format(api_key)}
#     response = requests.get(url, headers=headers, params=url_params)
#     num = response.json()['total']
#     print('{} total matches found.'.format(num))
#     cur = 0
#     results = []
#     while cur < num and cur < 1000:
#         url_params['offset'] = cur
#         results.append(yelp_call(url_params, api_key))
#         # function for getting specific variable and load to AWS can be used here
#         time.sleep(1) #Wait a second
#         cur += 50
#     return results


# all_bars = all_results(url_params, api_key)
# print(len(all_bars))

# all_bars[0][0]
# df = pd.DataFrame(all_bars)
# df.head()

['https://api.yelp.com/v3/businesses/xZwqtw3-dd6yJyixM-BYIA/reviews',
 'https://api.yelp.com/v3/businesses/RemvTpxalDjSew5HwmNzUQ/reviews',
 'https://api.yelp.com/v3/businesses/H8FodkT7iwLR69gNJiES7Q/reviews',
 'https://api.yelp.com/v3/businesses/2DJjJu3sqaBCN7w4CwR5Dw/reviews',
 'https://api.yelp.com/v3/businesses/OgM6WwztLVavBQBmb_g5aw/reviews',
 'https://api.yelp.com/v3/businesses/H8mnryiCI02N00g9vuFDLg/reviews',
 'https://api.yelp.com/v3/businesses/9sU8ceKHiqNrLBMRM-aUkg/reviews',
 'https://api.yelp.com/v3/businesses/_Tn0eN5E37WBf8oVdOA9BA/reviews',
 'https://api.yelp.com/v3/businesses/xEF3Kvd0yw74pjnlFB2Sgg/reviews',
 'https://api.yelp.com/v3/businesses/gjwELZmHUNO0WdqXYBEbcg/reviews',
 'https://api.yelp.com/v3/businesses/c3Ww3pgeHjBi3ctkeUWrjQ/reviews',
 'https://api.yelp.com/v3/businesses/eTR6nfHvQzvKX7Dcd6JrfA/reviews',
 'https://api.yelp.com/v3/businesses/NVx7gfdd4NDAwvj6URZ2aQ/reviews',
 'https://api.yelp.com/v3/businesses/zWrZBXj5F2V_0UknnTfqbw/reviews',
 'https://api.yelp.c

In [48]:
# write a function to insert the parsed data into the reviews table
# https://api.yelp.com/v3/businesses/{xZwqtw3-dd6yJyixM-BYIA}/reviews
url = 'https://api.yelp.com/v3/businesses/xZwqtw3-dd6yJyixM-BYIA/reviews'
headers = {'Authorization': 'Bearer {}'.format(api_key)}
response = requests.get(url, headers=headers)
data = response.json()['reviews']
data

[{'id': 'jLUPtLK0uOohN8WYpmynOg',
  'url': 'https://www.yelp.com/biz/the-pub-stranraer?adjust_creative=oXRz7HWebFp6YdQ0YT3PMQ&hrid=jLUPtLK0uOohN8WYpmynOg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=oXRz7HWebFp6YdQ0YT3PMQ',
  'text': 'Good place to grab a pint before the late boat!',
  'rating': 4,
  'time_created': '2015-11-28 10:38:33',
  'user': {'id': 'dt19J-gCZAELg6v9GmcrBA',
   'profile_url': 'https://www.yelp.com/user_details?userid=dt19J-gCZAELg6v9GmcrBA',
   'image_url': None,
   'name': 'Dave K.'}},
 {'id': '14jX8L6Y4baSkmfc4_u_4Q',
  'url': 'https://www.yelp.com/biz/the-pub-stranraer?adjust_creative=oXRz7HWebFp6YdQ0YT3PMQ&hrid=14jX8L6Y4baSkmfc4_u_4Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=oXRz7HWebFp6YdQ0YT3PMQ',
  'text': 'Now serving food homemade with produce sourced from local butchers fish mongers & farmers &\nLive Music Every Thursday Night great night out',
  'rating': 5,
  'time_created': '2009-02-04 09:04:15',

## Part 4: Write SQL queries that will answer the questions posed

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter.***

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results.append(yelp_call(url_params, api_key))
        # function for getting specific variable and load to AWS can be used here
        time.sleep(1) #Wait a second
        cur += 50
    return df

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id INT AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```